# Metodos en python para extracción de datos

Codigo en python el cual realiza la conexion a base de datos y exporta en formato excel los datos historicos de despacho mediante la función obtener_despachos, ademas mediante la función obtener_eventos_rastreo_gps podemos traer la data de un vehiculo en especifico con su detalle operativo


In [1]:
pip install mysql-connector-python==8.0.33

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.4/27.4 MB 51.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 11.5 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.25.6
    Uninstalling protobuf-4.25.6:
      Successfully uninstalled protobuf-4.25.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grpcio-status 1.62.3 requires protobuf>=4.21.6, but you have protobuf 3.20.3 which is incompatible.
tensorflow-metadata 1.16.1 requires protobuf<6.0.0dev,>=4.25.2; python_version >= "3.11", but you have protobuf 3.20.3 which is incompatible.


In [1]:
import mysql.connector
from mysql.connector import Error
import os
import csv

# Función para conectar a la base de datos
def conectar_bd(database):
    try:
        conexion = mysql.connector.connect(
            host=os.getenv("DB_HOST", "147.93.47.161"),
            user=os.getenv("DB_USER", "masterbus"),
            password=os.getenv("DB_PASSWORD", "9@#$%&yhif"),
            database=database
        )
        if conexion.is_connected():
            print(f"✅ Conexión a la base de datos {database} establecida.")
            return conexion
    except Error as e:
        print(f"❌ Error al conectar a MySQL ({database}): {e}")
        return None

# Función para obtener el esquema de la base de datos
def obtener_esquema(conexion):
    if not conexion:
        print("❌ No hay conexión a la base de datos.")
        return None

    try:
        cursor = conexion.cursor()
        query = """
            SELECT TABLE_NAME, COLUMN_NAME, DATA_TYPE, COLUMN_TYPE
            FROM INFORMATION_SCHEMA.COLUMNS
            WHERE TABLE_SCHEMA = %s
        """
        cursor.execute(query, (conexion.database,))
        resultado = cursor.fetchall()

        esquema = {}
        for tabla, columna, tipo, columna_tipo in resultado:
            if tabla not in esquema:
                esquema[tabla] = []
            esquema[tabla].append({"columna": columna, "tipo": tipo, "detalle": columna_tipo})

        cursor.close()
        return esquema
    except Error as e:
        print(f"❌ Error al obtener el esquema: {e}")
        return None

# Función para obtener despachos y exportarlos a CSV
# Genera una archivo CSV con los despachos entre las fechas indicadas
# Los campos exportados son: fecha_inicial, hora_inicial_plan, hora_inicial_real, hora_inicial_aux, fecha_final,
# hora_final_plan, hora_final_real, hora_final_aux, fk_ruta, pasajeros, distancia, fk_vehiculo, fk_conductor,
# estado_despacho, pk_intervalo_despacho, pk_informacion_registradora
# Los parámetros son:
# - conexion: la conexión a la base de datos
# - fecha_ini: fecha inicial para filtrar los despachos
# - fecha_fin: fecha final para filtrar los despachos
# - archivo_csv: nombre del archivo CSV donde se exportarán los despachos
# Retorna una lista con los despachos obtenidos o None si no se pudo obtener la información
# En donde cada columna del archivo CSV corresponde a:
# - FECHA_INICIAL Fecha real del despacho
# - HORA_INICIAL_PLAN Hora planificada de inicio del despacho
# - HORA_INICIAL_REAL Hora real de inicio del despacho
# - HORA_INICIAL_AUX Hora de inicio del despacho obtenida de la información registradora
# - FECHA_FINAL Fecha real de finalización del despacho
# - HORA_FINAL_PLAN Hora planificada de finalización del despacho
# - HORA_FINAL_REAL Hora real de finalización del despacho
# - HORA_FINAL_AUX Hora de finalización del despacho obtenida de la información registradora
# - FK_RUTA Llave foránea de la ruta
# - PASAJEROS Número de pasajeros
# - DISTANCIA Distancia recorrida
# - FK_VEHICULO Llave foránea del vehículo
# - FK_CONDUCTOR Llave foránea del conductor
# - ESTADO_DESPACHO Estado del despacho cuyos valores posibles son: 0 No controlado, 2 Controlado (Adelantado ó Atrasado), 3  Controlado (En tiempo permitido / holgura), 4 Terminado manualmente
# - PK_INTERVALO_DESPACHO Llave primaria del intervalo de despacho
def obtener_despachos(conexion, fecha_ini, fecha_fin, archivo_csv="despachos.csv"):
    if not conexion:
        print("❌ No hay conexión a la base de datos.")
        return None

    try:
        cursor = conexion.cursor(dictionary=True)
        query = """
            SELECT FECHA_INICIAL, HORA_INICIAL_PLAN, HORA_INICIAL_REAL, HORA_INICIAL_AUX, FECHA_FINAL,
                   HORA_FINAL_PLAN, HORA_FINAL_REAL, HORA_FINAL_AUX, FK_RUTA, PASAJEROS, DISTANCIA,
                   FK_VEHICULO, FK_CONDUCTOR, ESTADO_DESPACHO, PK_INTERVALO_DESPACHO, PK_INFORMACION_REGISTRADORA
            FROM
            (
                SELECT PK_INTERVALO_DESPACHO, FECHA_INICIAL, HORA_INICIAL AS HORA_INICIAL_PLAN,
                       (SELECT HORA_REAL FROM tbl_planilla_despacho WHERE FK_INTERVALO_DESPACHO = PK_INTERVALO_DESPACHO AND TIPO_PUNTO = 1) AS HORA_INICIAL_REAL,
                       FECHA_FINAL, HORA_FINAL AS HORA_FINAL_PLAN,
                       (SELECT HORA_REAL FROM tbl_planilla_despacho WHERE FK_INTERVALO_DESPACHO = PK_INTERVALO_DESPACHO AND TIPO_PUNTO = 3) AS HORA_FINAL_REAL,
                       (SELECT ESTADO_DESPACHO FROM tbl_planilla_despacho WHERE FK_INTERVALO_DESPACHO = PK_INTERVALO_DESPACHO AND TIPO_PUNTO = 3) AS ESTADO_DESPACHO
                FROM tbl_intervalo_despacho
                WHERE FECHA_INICIAL BETWEEN %s AND %s AND ESTADO = 1
            ) AS ta
            INNER JOIN
            (
                SELECT HORA_SALIDA_BASE_SALIDA AS HORA_INICIAL_AUX, HORA_INGRESO AS HORA_FINAL_AUX, FK_RUTA,
                       DIFERENCIA_NUM AS PASAJEROS, DISTANCIA_METROS AS DISTANCIA, FK_VEHICULO, FK_CONDUCTOR,
                       FK_INTERVALO_DESPACHO, PK_INFORMACION_REGISTRADORA
                FROM tbl_informacion_registradora
                WHERE FECHA_SALIDA_BASE_SALIDA BETWEEN %s AND %s
            ) AS tb
            ON ta.PK_INTERVALO_DESPACHO = tb.FK_INTERVALO_DESPACHO
        """
        cursor.execute(query, (fecha_ini, fecha_fin, fecha_ini, fecha_fin))
        resultados = cursor.fetchall()

        if not resultados:
            print("⚠️ No hay registros de despachos para exportar.")
            return None

        # Guardar resultados en CSV
        with open(archivo_csv, mode='w', newline='', encoding='utf-8') as file:
            writer = csv.DictWriter(file, fieldnames=resultados[0].keys())
            writer.writeheader()
            writer.writerows(resultados)

        print(f"✅ Datos exportados en '{archivo_csv}'.")
        cursor.close()
        return archivo_csv
    except Error as e:
        print(f"❌ Error al obtener los despachos: {e}")
        return None

# Función para obtener eventos de rastreo GPS y exportarlos a CSV con nombre dinámico
# Genera un archivo CSV con los eventos de rastreo GPS entre las fechas indicadas
def obtener_eventos_rastreo_gps(conexion_gps, conexion_principal, fecha_ini, fecha_fin, pk_vehiculo):
    if not conexion_gps or not conexion_principal:
        print("❌ No hay conexión a la base de datos.")
        return None

    try:
        cursor_principal = conexion_principal.cursor(dictionary=True)

        # Obtener la placa del vehículo
        query_placa = "SELECT PLACA FROM bd_montebello_rdw.tbl_vehiculo WHERE PK_VEHICULO = %s"
        cursor_principal.execute(query_placa, (pk_vehiculo,))
        resultado_placa = cursor_principal.fetchone()

        if not resultado_placa:
            print(f"⚠️ No se encontró la placa para el vehículo con PK_VEHICULO = {pk_vehiculo}.")
            cursor_principal.close()
            return None

        placa = resultado_placa["PLACA"]
        cursor_principal.close()

        cursor_gps = conexion_gps.cursor(dictionary=True)
        query_gps = """
            SELECT fecha_server, fecha_gps, trans_reason, msg, alarma AS cantidad_alarma,
                   distancia_metros, (@suma_distancia := @suma_distancia + distancia_metros) AS distancia_acumulada,
                   numeracion, total_dia, entradas, salidas, velocidad, latitud, longitud, placa, %s AS PK_VEHICULO
            FROM bd_montebello_rdw_gps.tbl_forwarding_wtch, (SELECT @suma_distancia := 0) AS var
            WHERE fecha_gps BETWEEN %s AND %s
            AND placa = %s
            ORDER BY fecha_gps ASC
        """

        cursor_gps.execute(query_gps, (pk_vehiculo, fecha_ini, fecha_fin, placa))
        resultados = cursor_gps.fetchall()

        if not resultados:
            print("⚠️ No hay registros de rastreo GPS para exportar.")
            cursor_gps.close()
            return None

        # Guardar en CSV con nombre dinámico
        archivo_csv = f"{placa}_FV_{pk_vehiculo}_rastreo_gps.csv"
        with open(archivo_csv, mode='w', newline='', encoding='utf-8') as file:
            writer = csv.DictWriter(file, fieldnames=resultados[0].keys())
            writer.writeheader()
            writer.writerows(resultados)

        print(f"✅ Datos exportados en '{archivo_csv}'.")
        cursor_gps.close()
        return archivo_csv
    except Error as e:
        print(f"❌ Error al obtener eventos GPS: {e}")
        return None

# Funcion que obtiene el listado de vehiculos
def obtener_listado_vehiculos(conexion):
    if not conexion:
        print("❌ No hay conexión a la base de datos.")
        return None
    try:
        cursor = conexion.cursor(dictionary=True)
        query = """
            SELECT * FROM bd_montebello_rdw.tbl_vehiculo
        """
        cursor.execute(query)
        resultados = cursor.fetchall()
        if not resultados:
            print("⚠️ No hay registros de vehiculos para exportar.")
            return None
        archivo_csv = "listado_vehiculos.csv"
        with open(archivo_csv, mode='w', newline='', encoding='utf-8') as file:
            writer = csv.DictWriter(file, fieldnames=resultados[0].keys())
            writer.writeheader()
            writer.writerows(resultados)
        print(f"✅ Datos exportados en '{archivo_csv}'.")
        cursor.close()
        return archivo_csv
    except Error as e:
        print(f"❌ Error al obtener los vehiculos: {e}")
        return None

# Funcion que optiene todos los eventos de rastreo dada un rago de fechas
def obtener_eventos_rastreo(conexion, fecha_ini, fecha_fin):
    if not conexion:
        print("❌ No hay conexión a la base de datos.")
        return None
    try:
        cursor = conexion.cursor(dictionary=True)
        query = """
            SELECT * FROM bd_montebello_rdw_gps.tbl_forwarding_wtch
            WHERE fecha_gps BETWEEN %s AND %s
        """
        cursor.execute(query, (fecha_ini, fecha_fin))
        resultados = cursor.fetchall()
        if not resultados:
            print("⚠️ No hay registros de rastreo para exportar.")
            return None
        archivo_csv = "eventos_rastreo.csv"
        with open(archivo_csv, mode='w', newline='', encoding='utf-8') as file:
            writer = csv.DictWriter(file, fieldnames=resultados[0].keys())
            writer.writeheader()
            writer.writerows(resultados)
        print(f"✅ Datos exportados en '{archivo_csv}'.")
        cursor.close()
        return archivo_csv
    except Error as e:
      print(f"❌ Error al obtener los eventos de rastreo: {e}")
    return None

# Función para cerrar la conexión a la base de datos
def cerrar_conexion(conexion):
    if conexion and conexion.is_connected():
        conexion.close()
        print("✅ Conexión cerrada correctamente.")

# Función principal
if __name__ == "__main__":
    conexion_principal = conectar_bd("bd_montebello_rdw") # Consolidados de rutas y despachos ,, resumen
    conexion_gps = conectar_bd("bd_montebello_rdw_gps") # neventos de vehiculos

    if conexion_principal and conexion_gps:
        # Obtener esquemas de las bases de datos ---  Solo si es necesario
        # esquema_principal = obtener_esquema(conexion_principal)
        # esquema_gps = obtener_esquema(conexion_gps)

        # Obtener datos de despachos ----  Lo siguiente a manera de ejemplo
        # obtener_listado_vehiculos(conexion_principal)
        fecha_ini = '2024-04-16'
        fecha_fin = '2025-03-03'
        obtener_despachos(conexion_principal, fecha_ini, fecha_fin)
        #obtener_eventos_rastreo(conexion_gps, fecha_ini, fecha_fin)
        #obtener_eventos_rastreo_gps(conexion_gps, conexion_principal, '2024-04- 08:59:00', '2024-04-15 12:00:06', 9)

    cerrar_conexion(conexion_principal)
    cerrar_conexion(conexion_gps)


✅ Conexión a la base de datos bd_montebello_rdw establecida.
✅ Conexión a la base de datos bd_montebello_rdw_gps establecida.
✅ Datos exportados en 'despachos.csv'.
✅ Conexión cerrada correctamente.
✅ Conexión cerrada correctamente.
